#Differential Privacy

##Motivation

An example (Quoting Paul Ohm):

In the 1990s, an insurance company released 'anonymised' health data on all state employees in Massachusetts. The Governor, said that GIC had protected patient privacy by deleting identifiers. Latanya Sweeney (a postgrad researcher) showed otherwise. 

How it was done:
 - She knew that Governor Weld resided in Cambridge, Massachusetts, a city of 54,000 residents and seven ZIP codes.
 - For twenty dollars, she purchased the complete voter rolls from the city of Cambridge, a database containing, among other things, the name, address, ZIP code, birth date, and sex of every voter.
 - By combining this data with the GIC records, Sweeney found Governor Weld with ease.
   - Only six people in Cambridge shared his birth date
   - Only three of them men
   - Of them, only he lived in his ZIP code.
   
In a theatrical flourish, Dr. Sweeney sent the Governor’s health records (which included diagnoses and prescriptions) to his office[1]

##Census Data

Another example, UK census data.

An example table for my "Output Area".

|  | Males |  |  |  |  |  |  | Females |  |  |  |  |  |  |
|------------|---------|----------|----------|----------|----------|----------|-------------|---------|----------|----------|----------|----------|----------|-------------|
|  | 0 to 15 | 16 to 24 | 25 to 34 | 35 to 49 | 50 to 64 | 65 to 74 | 75 and over | 0 to 15 | 16 to 24 | 25 to 34 | 35 to 49 | 50 to 64 | 65 to 74 | 75 and over |
| Christian | 1 | 20 | 11 | 8 | 2 | 0 | 1 | 0 | 15 | 14 | 2 | 0 | 1 | 0 |
| Buddhist | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| Hindu | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| Jewish | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| Muslim | 3 | 6 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 2 | 0 | 0 | 0 |
| Sikh | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
| Other | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 2 | 0 | 1 | 0 | 0 | 0 | 0 |
| None | 5 | 35 | 34 | 12 | 6 | 1 | 0 | 4 | 18 | 22 | 7 | 2 | 1 | 0 |
| Not Stated | 1 | 5 | 1 | 4 | 0 | 0 | 0 | 1 | 0 | 2 | 0 | 1 | 0 | 0 |

In [237]:
a[1,2]

array([ 0.49444444,  0.00555556,  0.00555556,  0.01666667,  0.00555556,
        0.00555556,  0.02222222,  0.44444444])

In [239]:
mat[0][0][1][2]


array([ 0.49444444,  0.00555556,  0.00555556,  0.01666667,  0.00555556,
        0.00555556,  0.02222222,  0.44444444])

In [110]:
#white = data.ix[0,398+(4*6):(398+(6*11)):1][1*6:4*6]
#data.ix[0,2:2+6*4]
#white

values = data.ix[0,:]
res = {}
for col,v in zip(data.columns,values):
    c = col.split('~')
    if (len(c)>1):
        temp = res
        for ix in range(len(c)):
            if c[ix] in temp:
                temp = temp[c[ix]]
            else:
                if ix==len(c)-1:
                    temp[c[ix]] = v
                else:
                    temp[c[ix]] = {}
                    temp = temp[c[ix]]
                
                
        

In [121]:
res['Females']['Age 16 to 49']['Day-to-day activities limited a lot']['Bad or very bad health']['Asian/Asian British']

1

###Introduction

> differential privacy: "maximize the accuracy of queries from statistical databases while minimizing the chances of identifying its records." (wikipedia)

> differential privacy: "a promise by a curator to a data subject that they will not be affected by being included, regardless what other sources of data are available" (privacy book, The Algorithmic Foundations of Differential Privacy - Cynthia Dwork & Aaron Roth).

This brings in the idea of a "linkage attack", combining data sources to infer details about individuals in an "anonymised" dataset.

From Dwork & Roth again:
> "[DP]... ensures that any [output] is “essentially” equally likely to occur, independent of the presence or absence of any
individual. The term “essentially” is captured by a parameter, $\varepsilon$. A smaller $\varepsilon$ will yield better privacy (and less accurate responses).

And a list of hints/rules-of-thumb (again from Dwork & Roth, actually most of this document is based on their book or wikipedia, which is also based on their book. Maybe just go read the book!).

 - Data cannot be fully anonymised and remain useful.
 - Reidentification of anonymised records is not the only risk.
 - Queries over large sets are not protective.
 - Query auditing is problematic (e.g. refusing a query could be disclosive, computationally infeasible).
 - Summary statistics are not safe.
 - "Ordinary facts" are not ok (e.g. shopping habits - give clues around health, household, religion, etc).
 - Cancel privacy for "just a few" people, for "greater good" (outliers, etc): A possible moral standpoint, but which is usually (reasonably) rejected.

A widely used example of Differential Privacy (before it was called that), is the coin-flip example. Participants being asked if they take part in illegal or socially-unacceptable behaviour will often lie. Sociologists have avoided this problem by asking people to flip a coin: If tails they should report the truth, but if heads, the participant must flip again and report yes if heads, no if tails.

![alt text](files/images/diff_priv_coin_small.png "Coin Flip")

Effectively provides 'plausible deniability'.

***Any nontrivial privacy guarantee that holds, regardless of all present and future sources of auxiliary information requires RANDOMISATION***

###A simple example

Using the wikipedia example, and extending/explaining it.

If we have a database, such as this one:
    
    Name    Has Diabetes
    0 Adam         0
    1 Beth         1
    2 Charlie      1
    3 Diane        0
    4 Elizabeth    0
    5 Fred         1 (new person)

We want to provide a summary statistic (such as the sum number of people with diabetes: Maybe to know how many drugs to stock).

If a new person is added to the database, we could query it again. Unfortunately by subtracting the two values we could find his diabetes-state, even though the query result itself doesn't mention names.

Differential Privacy aims to avoid that. Before releasing the summary value, one must apply a function to the value to provide this privacy. This is usually a matter of adding a random value. For this example, if we are first asked for the sum (without Fred), we take the actual sum (2) and add/subtract a value (e.g. +1.6): Giving 3.6 as the sum. 

When Fred is added and someone asked for the sum, we do the same: Take the actual sum (3) and add/subtract a random value (e.g. +0.9), giving 3.9.

The adversary, who asked these two queries can't be sure whether Fred has diabetes.

###More formal definition

Slightly more formally:

We have a randomisation algorithm $M$. It is $\varepsilon$-private if the following holds:

If we consider the outputs $M(D_1)$ and $M(D_2)$ provided to the adversary, from the database with ($D_1$) and without ($D_2$) Fred, we can compare the probability of any given value output $P[M(D_1)]$ and $P[M(D_2)]$. We want the probabilities of these two outputs to be very similar, such that, for any value of M:

$P[M(D_1)] \leq e^{\varepsilon} P[M(D_2)] + \delta$

Note: Often the two databases are defined by their $L_1$ norm, such that: $||D_1 - D_2|| \leq 1$. Where ||D|| is the sum over the number of elements of each type. Basically the length of the database (?). **TBC**

####Delta: $\delta$

If $\delta=0$ we say that the algorithm is $\varepsilon$-differentially private.

For example,...? TODO...

###Coin flip example continued...

From http://www.cs.cmu.edu/afs/cs/academic/class/15859m-s11/www/lectures/lect0420.pdf

It is possibly clearer to write the above inequality as:

$e^{-\varepsilon} \leq {{P[M(X)=v]} \over {P[M(X')=v]}} \leq e^{\varepsilon}$

where $X$ and $X'$ are the two databases, with one row changed, where the change is the one with the largest possible effective on the value of $M(X)$.

The coin flip above, the probability of an outcome (e.g. a Yes) given the underlying 'database'/actual value is 'True', is ${}^3/_4$. Similarly the probability of the same outcome given the most different underlying actual value (False) is, ${}^1/_4$. Substituting into the above equation: 

$e^{-\varepsilon} \leq {{3/4} \over {1/4}} \leq e^{\varepsilon}$

$3 = e^{\varepsilon}$

$\varepsilon = ln(3)$

So we can say that this algorithm is $(ln\; 3, 0)$-differentially private.

####Gaussian output perterbation and Laplace mechanism

The obvious general solution is to add some gaussian noise to the output. Consider $n$ rows, with a column of values between 0 and $\Delta f$, where we set $\Delta f$ as 40k for this example. Note: $\Delta f$ is known, in general as the $l_1$-sensitivity of the function, and it's how different two database outputs could be, given the databases are neighbours.
    
    Person  Income/£k
    Amy       20
    Bob       17
    Charlie   23
    Diana     37
    Elsbeth   26
    Fred      22
    Gavin     17
    Holly     29

We want to find the total income. The largest an individual might change the mean is 40k. We want to add sufficient noise to sufficiently mask this change. If we first consider adding Gaussian noise, then the ratio of the two probabilities is:

${exp[{{{-x^2} \over {2\sigma^2}}}]
\over
{exp[{{-(x-[\Delta f])^2} \over {2\sigma^2}}]}}$

which can be rearranged and cancelled through:

${exp[{{{-x^2} \over {2\sigma^2}}}]
\over
{exp[{{-(x-[\Delta f])^2} \over {2\sigma^2}}]}}$

$exp[{{-x^2 + (x-[b])^2} \over {2 \sigma^2}}] = exp[{{-2(\Delta f)x+{\Delta f}^2} \over {2 \sigma^2}}]$

$\Delta f^2$ is almost zero, so ignoring that term:

$e^-\varepsilon = exp[{{-2(\Delta f)x} \over {2 \sigma^2}}]$

$\varepsilon = {2\Delta f x \over {2 \sigma^2}}$

Extra $x$ might be a problem?? Use Laplace instead!

####Laplace Mechanism

$exp[-\varepsilon] = {{exp[|x|/b}] \over {exp[|x-{\Delta f}|\;/\;b]}}$

$exp[-\varepsilon] = exp[{-{\Delta f} \over {b}}]$

$\varepsilon = {{\Delta f} \over {b}}$

So we set the parameter $b$ equal to ${\Delta f} / {\varepsilon}$.

So in our example from earlier, $\Delta f=40$. We finally need to select a value for $\varepsilon$ which balances the privacy of the members of database and the utility researchers can gain from the database (a smaller value gives more privacy). In this example let's set $\varepsilon$ to 1: $b = 40$.

####Privacy loss
[to complete]
How much privacy is lost? The log of the probability ratio is known as the *privacy loss*, incurred by observing a result, E.

$L^{(E)}_{M(x)||M(x')} = ln \left( {{P[M(x)=E]} \over {P[M(x')=E]}} \right)$

The absolute value of the privacy loss is bounded by $\varepsilon$ (with probability $1-\delta$).

###Theorems (summarised into English a bit)

- If several queries are made $(\varepsilon_1,\delta_1)$, $(\varepsilon_2,\delta_2)$,... then the $\varepsilon$s and $\delta$s add up.

- If several rows are of concern ("group privacy"), for query of $(\varepsilon,0)$-DP the same query is only $(k\varepsilon,0)$-DP for $k$ rows.

####Counting Queries...

"How many people have an income over £25?"

The sensitivity of a counting query, $\Delta f = 1$ (as each row can only change the result by a maximum of 1. We therefore simply add Laplace noise with parameter $b = 1/\varepsilon$. If $\varepsilon=1$, then $b=1$.

In [5]:
import numpy as np
income = np.array([20,17,23,37,26,22,17,29])
np.sum(income>25)+np.random.laplace(0,1)

2.3767712043304892

If 5 counting queries are asked, we need to use $\varepsilon_x = \varepsilon / 5$ for each query to ensure $\varepsilon$ privacy overall. $b = \Delta f / \varepsilon_x = 1/5$

In [28]:
import numpy as np
income = np.array([20,17,23,37,26,22,17,29])
print np.sum(income>35)+np.random.laplace(0,5)
print np.sum((income>25) & (income<=35))+np.random.laplace(0,5)
print np.sum((income>15) & (income<=25))+np.random.laplace(0,5)
print np.sum((income>5) & (income<=15))+np.random.laplace(0,5)
print np.sum(income<=5)+np.random.laplace(0,5)

-1.7240892506
-6.84852639568
15.1490919063
0.87224599347
8.83005847491


In [8]:
np.sum(income)+np.random.laplace(0,40)

169.78594073894317

####Histogram Queries...
Disjoint cells: Can be more efficient than just a series of seperate counts as the addition of one row can only affect one cell. Means, as long as the 5 cells are disjoint we can just use $\varepsilon$.

####Report noisy max
To do

###Applying to individual collections of data

To rephrase the above to our problem: What effect does adding a data point $x$ have to the summary value? We need to know the sensitivity of the algorithm. Questions come up about normalisation (to normalise we need to know the values of the variables!).




[1] http://arstechnica.com/tech-policy/2009/09/your-secrets-live-online-in-databases-of-ruin/
